<img src="https://pollinations.ai/ipfs/QmRs28NiwU13SEmTCPoLFktQWsdoDSXYeay5bQBPFtrQ3M" width="300" height="300" />

GFPGAN is a blind face restoration algorithm towards real-world face images. 
It leverages the generative face prior in a pre-trained GAN (*e.g.*, StyleGAN2) to restore realistic faces while precerving fidelity. 



In [ ]:
# Input image of face that will be restored
file_image = '' #@param {type: "string"}

# Choose whether to colorize the image or not. The model used is slightly different. You may get higher quality results without colorization.
colorize = False #@param {type: "boolean"}

output_path = '/content/output'

In [ ]:
image_path = '/tmp/images'
!mkdir -p {image_path}
!mkdir -p {output_path}
!cp -v {file_image} {image_path}

# Clone GFPGAN and enter the GFPGAN folder
%cd /content
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install ninja
!pip install git+https://github.com/xinntao/BasicSR.git
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions

# Download the pre-trained model
if colorize:
    !wget https://github.com/TencentARC/GFPGAN/releases/download/v0.1.0/GFPGANv1.pth -P experiments/pretrained_models
else:
    !wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models


# 2. Upload Images / Use the demo images

### OR you can use the demo image by running the following codes

In [ ]:
tmp_output_path = "/tmp/output_gfpgan"
!mkdir -p $tmp_output_path
!rm -rf $tmp_output_path/*

# Now we use the GFPGAN to restore the above low-quality images
# We use [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) for enhancing the background (non-face) regions

if colorize:
    !BASICSR_JIT=True python inference_gfpgan.py --model_path experiments/pretrained_models/GFPGANv1.pth  --test_path {image_path} --save_root {tmp_output_path} --arch original --channel 1
else:
    !BASICSR_JIT=True python inference_gfpgan.py --upscale 2 --test_path {image_path} --save_root {tmp_output_path} --model_path experiments/pretrained_models/GFPGANCleanv1-NoCE-C2.pth --bg_upsampler realesrgan

!cp -v {tmp_output_path}/cropped_faces/* {output_path}/photo_1_cropped.png
!cp -v {tmp_output_path}/restored_faces/* {output_path}/photo_2_restored.png
!cp -v {tmp_output_path}/cmp/* {output_path}/photo_3_comparison.png
!cp -v {tmp_output_path}/restored_imgs/* {output_path}/photo_4_restored.png

!sleep 5